#TECH CHALLENGE - FASE 4

Patrícia Vieira Abraham | Fillipe Júlio de Oliveira Nascimento

# Análise de Regressão Logística para Previsão do Ibovespa

Este notebook tem como objetivo desenvolver e avaliar um modelo de regressão logística para prever a direção do fechamento do Ibovespa no dia seguinte. Especificamente, o modelo tentará prever se o preço de fechamento do Ibovespa no dia t+1 será maior que o preço de fechamento no dia t.

### Visão Geral do Pipeline
O pipeline de machine learning implementado neste notebook segue as seguintes etapas:
1.  Carregamento e Pré-processamento dos Dados: Leitura do arquivo CSV do Ibovespa, parse de datas, ordenação temporal e limpeza inicial dos dados.
2.  Engenharia de Atributos: Criação de diversas features técnicas e temporais a partir dos dados brutos, como retornos, médias móveis, volatilidades e indicadores de bandas.
3.  Definição da Variável Target: Criação da variável binária que indica se o fechamento do dia seguinte foi maior que o fechamento atual.
4.  Divisão Temporal dos Dados: Separação dos dados em conjuntos de treino e teste, respeitando a ordem cronológica para evitar vazamento de dados.
5.  Construção e Treinamento do Pipeline: Definição de um sklearn.pipeline.Pipeline que inclui um StandardScaler para normalização dos atributos e um modelo de LogisticRegression.
6.  Validação Cruzada Temporal: Avaliação do desempenho do pipeline usando TimeSeriesSplit no conjunto de treino.
7.  Treinamento Final: Treinamento do pipeline completo no conjunto de treino.
8.  Avaliação do Modelo: Geração de predições e avaliação das métricas de desempenho (acurácia, precisão, recall, F1-score, ROC AUC, matriz de confusão, relatório de classificação) no conjunto de teste.
9.  Exportação de Artefatos: Salvamento do pipeline treinado, das métricas de avaliação e das predições do conjunto de teste em formatos pickle, json e csv, respectivamente.

###Instruções de Execução

Para executar este notebook, siga os passos abaixo:
    
1.  Pré-requisitos: Certifique-se de ter as bibliotecas Python necessárias instaladas (pandas, numpy, scikit-learn, joblib).
Você pode instalá-las via pip:
    
    !pip install pandas numpy scikit-learn joblib
    
2.  Estrutura de Pastas: Crie uma pasta chamada data na mesma raiz onde este notebook está salvo. Dentro da pasta data, coloque o arquivo Ibovespa.csv.
3.  Execução: Execute todas as células do notebook em ordem. Ao final da execução, uma pasta artifacts será criada (se não existir) contendo:
    -   logreg_pipeline.pkl: O pipeline de machine learning treinado.
    -   metrics.json: As métricas de avaliação do modelo no conjunto de teste.
    -   test_predictions.csv: As predições do modelo para o conjunto de teste, incluindo a variável real, a predição e a probabilidade.
    -   pipeline_execution.log: Um arquivo de log com informações sobre a execução do pipeline.

In [46]:
import os
import logging
import json
import joblib
import pandas as pd
import numpy as np

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    confusion_matrix,
    classification_report
)


# --- Configurações Globais ---
DATA_PATH = "data/Dados Históricos - Ibovespa (5).csv"
ARTIFACTS_DIR = "artifacts"
LOG_FILE = os.path.join(ARTIFACTS_DIR, "pipeline_execution.log")
RANDOM_STATE = 42
TEST_SIZE = 30

# --- Configuração de Logging ---
def setup_logging(log_file: str):
    os.makedirs(os.path.dirname(log_file), exist_ok=True)

    # Get the logger for this module
    module_logger = logging.getLogger(__name__)
    module_logger.setLevel(logging.INFO) # Set the minimum level for this logger

    # Clear existing handlers to prevent duplicate output if run multiple times
    if module_logger.handlers:
        for handler in list(module_logger.handlers): # Iterate over a copy to safely remove
            module_logger.removeHandler(handler)

    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')

    # File handler
    file_handler = logging.FileHandler(log_file, mode='w')
    file_handler.setLevel(logging.INFO)
    file_handler.setFormatter(formatter)
    module_logger.addHandler(file_handler)

    # Stream handler (console)
    stream_handler = logging.StreamHandler()
    stream_handler.setLevel(logging.INFO)
    stream_handler.setFormatter(formatter)
    module_logger.addHandler(stream_handler)

# Call the setup function
setup_logging(LOG_FILE)
# Get the logger instance for use in the rest of the notebook
logger = logging.getLogger(__name__)

# --- Funções Utilitárias ---
def _check_file_exists(file_path: str) -> None: #Verifica se um arquivo existe e levanta um erro se não.
    if not os.path.exists(file_path):
        logger.error(f'Erro: O arquivo {file_path} não foi encontrado. Certifique-se de que ele está na pasta "data".')
        raise FileNotFoundError(f'Arquivo não encontrado: {file_path}')

def _create_directory(path: str) -> None: #Cria um diretório se ele não existir.
    os.makedirs(path, exist_ok=True)
    logger.info(f' Diretório {path} garantido.')

def _calculate_bollinger_bands(df: pd.DataFrame, column: str, window: int = 20, num_std: int = 2) -> pd.DataFrame: #Calcula as Bandas de Bollinger.
    ma = df[column].rolling(window=window).mean()
    std = df[column].rolling(window=window).std()
    df[f'{column}_BB_Upper'] = ma + (std * num_std)
    df[f'{column}_BB_Lower'] = ma - (std * num_std)
    return df

def _calculate_atr(df: pd.DataFrame, window: int = 14) -> pd.DataFrame: #Calcula o Average True Range (ATR).
    high_low = df['Máxima'] - df['Mínima']
    high_prev_close = np.abs(df['Máxima'] - df['Último'].shift(1))
    low_prev_close = np.abs(df['Mínima'] - df['Último'].shift(1))
    tr = pd.DataFrame({'hl': high_low, 'hpc': high_prev_close, 'lpc': low_prev_close}).max(axis=1)
    df['ATR'] = tr.rolling(window=window).mean()
    return df

def _calculate_parkinson_volatility(df: pd.DataFrame, window: int = 20) -> pd.DataFrame: #Calcula a Volatilidade de Parkinson.
    # Parkinson Volatility = sqrt( (1 / (4 * ln(2))) * (ln(High/Low))^2 )
    # Usamos uma aproximação ou a forma mais comum de cálculo para dados diários
    # Aqui, vamos usar a diferença logarítmica entre máxima e mínima
    df['Parkinson_Vol'] = np.sqrt(0.5 * (np.log(df['Máxima'] / df['Mínima'])**2).rolling(window=window).mean())
    return df

def _engineer_features(df: pd.DataFrame) -> pd.DataFrame: #Cria atributos técnicos e temporais a partir dos dados brutos.
    logger.info('Iniciando engenharia de atributos...')
    df_copy = df.copy()

    # Retornos e Diferenças
    df_copy['pct_change_close'] = df_copy['Último'].pct_change()
    df_copy['diff_close'] = df_copy['Último'].diff()
    df_copy['high_low_diff'] = df_copy['Máxima'] - df_copy['Mínima']
    df_copy['open_close_diff'] = df_copy['Último'] - df_copy['Abertura']

    # Médias Móveis Simples (SMA)
    for window in [3, 7, 14, 21, 30]:
        df_copy[f'SMA_{window}'] = df_copy['Último'].rolling(window=window).mean()
        df_copy[f'SMA_Vol_{window}'] = df_copy['Var%'].rolling(window=window).mean()

    # Volatilidade (desvio padrão)
    for window in [7, 14, 21]:
        df_copy[f'Vol_Std_{window}'] = df_copy['Último'].rolling(window=window).std()

    # Indicadores Avançados
    df_copy = _calculate_parkinson_volatility(df_copy)
    df_copy = _calculate_atr(df_copy)
    df_copy = _calculate_bollinger_bands(df_copy, 'Último')

    # Atributos Temporais
    df_copy['day_of_week'] = df_copy['Data'].dt.dayofweek
    df_copy['day_of_month'] = df_copy['Data'].dt.day
    df_copy['month'] = df_copy['Data'].dt.month
    df_copy['year'] = df_copy['Data'].dt.year

    logger.info('Engenharia de atributos concluída.')
    return df_copy

def _evaluate_model(y_true: pd.Series, y_pred: np.ndarray, y_proba: np.ndarray) -> dict: #Calcula e retorna as principais métricas de avaliação do modelo.
    logger.info('Calculando métricas de avaliação...')
    metrics = {
        'accuracy_test': accuracy_score(y_true, y_pred),
        'precision_test': precision_score(y_true, y_pred),
        'recall_test': recall_score(y_true, y_pred),
        'f1_test': f1_score(y_true, y_pred),
        'roc_auc_test': roc_auc_score(y_true, y_proba),
        'confusion_matrix': confusion_matrix(y_true, y_pred).tolist(), # Conversão para lista no JSON
        'classification_report': classification_report(y_true, y_pred)
    }
    logger.info('Métricas calculadas com sucesso.')
    return metrics

## 1. Carregamento e Pré-processamento dos Dados

In [47]:
logger.info('Iniciando carregamento e pré-processamento dos dados.')
_check_file_exists(DATA_PATH)

try:
    df = pd.read_csv(DATA_PATH)
    logger.info(f'Arquivo {DATA_PATH} carregado com sucesso. Shape inicial: {df.shape}')

    # Parse de datas e ordenação
    df['Data'] = pd.to_datetime(df['Data'], format="%d.%m.%Y")
    df = df.sort_values('Data').reset_index(drop=True)
    logger.info('Coluna "Data" convertida e DataFrame ordenado temporalmente.')

    # Limpeza da coluna 'Var%'
    if 'Var%' in df.columns:
        df['Var%'] = df['Var%'].str.replace(',', '.', regex=False).str.replace('%', '', regex=False).astype(float) / 100
        logger.info('Coluna "Var%" limpa e convertida para float.')
    else:
        logger.warning('Coluna "Var%" não encontrada no DataFrame.')

    # Remoção da coluna 'Vol.'
    if 'Vol.' in df.columns:
        df = df.drop(columns=['Vol.'])
        logger.info('Coluna "Vol." removida.')
    else:
        logger.warning('Coluna "Vol." não encontrada no DataFrame.')

    logger.info(f'Pré-processamento inicial concluído. Shape atual: {df.shape}')
    logger.info('Primeiras 5 linhas do DataFrame após pré-processamento inicial:' + df.head().to_string())

except Exception as e:
    logger.error(f'Erro durante o carregamento ou pré-processamento inicial dos dados: {e}')
    raise

2026-01-16 03:03:45,387 - INFO - Iniciando carregamento e pré-processamento dos dados.
INFO:__main__:Iniciando carregamento e pré-processamento dos dados.
2026-01-16 03:03:45,402 - INFO - Arquivo data/Dados Históricos - Ibovespa (5).csv carregado com sucesso. Shape inicial: (4974, 7)
INFO:__main__:Arquivo data/Dados Históricos - Ibovespa (5).csv carregado com sucesso. Shape inicial: (4974, 7)
2026-01-16 03:03:45,421 - INFO - Coluna "Data" convertida e DataFrame ordenado temporalmente.
INFO:__main__:Coluna "Data" convertida e DataFrame ordenado temporalmente.
2026-01-16 03:03:45,429 - INFO - Coluna "Var%" limpa e convertida para float.
INFO:__main__:Coluna "Var%" limpa e convertida para float.
2026-01-16 03:03:45,432 - INFO - Coluna "Vol." removida.
INFO:__main__:Coluna "Vol." removida.
2026-01-16 03:03:45,434 - INFO - Pré-processamento inicial concluído. Shape atual: (4974, 6)
INFO:__main__:Pré-processamento inicial concluído. Shape atual: (4974, 6)
2026-01-16 03:03:45,441 - INFO - Pri

## 2. Engenharia de Atributos e Definição da Variável Target

In [48]:
logger.info('Iniciando engenharia de atributos e definição da variável target.')

try:
    # Criar atributos
    df_processed = _engineer_features(df.copy())

    # Criar variável target: 1 se o fechamento de amanhã for maior que o de hoje, 0 caso contrário
    df_processed['target'] = (df_processed['Último'].shift(-1) > df_processed['Último']).astype(int)
    logger.info('Variável "target" criada (1 se fechamento de amanhã > hoje, 0 caso contrário).')

    # Remover NaNs gerados pela engenharia de atributos e pelo shift da target
    initial_rows = df_processed.shape[0]
    df_processed.dropna(inplace=True)
    rows_dropped = initial_rows - df_processed.shape[0]
    logger.info(f'{rows_dropped} linhas com valores NaN foram removidas após engenharia de atributos e criação da target.')

    logger.info(f'Engenharia de atributos e target concluída. Shape final: {df_processed.shape}')
    logger.info('Colunas do DataFrame após engenharia de atributos:' + str(df_processed.columns.tolist()))
    logger.info('Primeiras 5 linhas do DataFrame após engenharia de atributos e remoção de NaNs:' + df_processed.head().to_string())

except Exception as e:
    logger.error(f'Erro durante a engenharia de atributos ou criação da target: {e}')
    raise

2026-01-16 03:04:35,858 - INFO - Iniciando engenharia de atributos e definição da variável target.
INFO:__main__:Iniciando engenharia de atributos e definição da variável target.
2026-01-16 03:04:35,864 - INFO - Iniciando engenharia de atributos...
INFO:__main__:Iniciando engenharia de atributos...
2026-01-16 03:04:35,887 - INFO - Engenharia de atributos concluída.
INFO:__main__:Engenharia de atributos concluída.
2026-01-16 03:04:35,890 - INFO - Variável "target" criada (1 se fechamento de amanhã > hoje, 0 caso contrário).
INFO:__main__:Variável "target" criada (1 se fechamento de amanhã > hoje, 0 caso contrário).
2026-01-16 03:04:35,897 - INFO - 29 linhas com valores NaN foram removidas após engenharia de atributos e criação da target.
INFO:__main__:29 linhas com valores NaN foram removidas após engenharia de atributos e criação da target.
2026-01-16 03:04:35,899 - INFO - Engenharia de atributos e target concluída. Shape final: (4945, 32)
INFO:__main__:Engenharia de atributos e target

## 3. Divisão Temporal dos Dados

In [49]:
logger.info('Iniciando divisão temporal dos dados em treino e teste.')

try:
    # Definir features (X) e target (y)
    # Excluir colunas que não são features ou são a própria target
    features_to_exclude = ['Data', 'Fechamento', 'Abertura', 'Máxima', 'Mínima', 'target']
    X = df_processed.drop(columns=[col for col in features_to_exclude if col in df_processed.columns])
    y = df_processed['target']

    # Armazenar a coluna 'Data' para o conjunto de teste para exportação futura
    data_test_series = df_processed['Data'].iloc[- TEST_SIZE:]

    # Divisão temporal (sem shuffle)
    train_size = int(len(df_processed) - TEST_SIZE)
    X_train, X_test = X.iloc[:train_size], X.iloc[train_size:]
    y_train, y_test = y.iloc[:train_size], y.iloc[train_size:]

    logger.info(f'Divisão de dados concluída. Base de teste: {TEST_SIZE}')
    logger.info(f'Shape de X_train: {X_train.shape}, y_train: {y_train.shape}')
    logger.info(f'Shape de X_test: {X_test.shape}, y_test: {y_test.shape}')
    logger.info('Primeiras 5 linhas de X_train:' + X_train.head().to_string())
    logger.info('Primeiras 5 linhas de y_train:' + y_train.head().to_string())

except Exception as e:
    logger.error(f'Erro durante a divisão temporal dos dados: {e}')
    raise

2026-01-16 03:04:57,219 - INFO - Iniciando divisão temporal dos dados em treino e teste.
INFO:__main__:Iniciando divisão temporal dos dados em treino e teste.
2026-01-16 03:04:57,524 - INFO - Divisão de dados concluída. Base de teste: 30
INFO:__main__:Divisão de dados concluída. Base de teste: 30
2026-01-16 03:04:57,526 - INFO - Shape de X_train: (4915, 27), y_train: (4915,)
INFO:__main__:Shape de X_train: (4915, 27), y_train: (4915,)
2026-01-16 03:04:57,530 - INFO - Shape de X_test: (30, 27), y_test: (30,)
INFO:__main__:Shape de X_test: (30, 27), y_test: (30,)
2026-01-16 03:04:57,544 - INFO - Primeiras 5 linhas de X_train:    Último    Var%  pct_change_close  diff_close  high_low_diff  open_close_diff      SMA_3  SMA_Vol_3      SMA_7  SMA_Vol_7     SMA_14  SMA_Vol_14     SMA_21  SMA_Vol_21     SMA_30  SMA_Vol_30  Vol_Std_7  Vol_Std_14  Vol_Std_21  Parkinson_Vol       ATR  Último_BB_Upper  Último_BB_Lower  day_of_week  day_of_month  month  year
29  25.536  0.0208          0.020787     

## 4. Construção e Treinamento do Pipeline

In [50]:
logger.info('Iniciando construção e treinamento do pipeline de machine learning.')

try:
    # Definir o pipeline: StandardScaler + LogisticRegression
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('classifier', LogisticRegression(
            solver='saga', # 'liblinear' ou 'saga' são bons para datasets menores e L1/L2
            max_iter=2000, # Aumentar max_iter para garantir convergência
            random_state=RANDOM_STATE,
            n_jobs=-1 # Usar todos os cores disponíveis
        ))
    ])
    logger.info('Pipeline definido: StandardScaler -> LogisticRegression.')

    # Validação cruzada com TimeSeriesSplit
    tscv = TimeSeriesSplit(n_splits=5) # 5 splits para validação temporal
    cv_scores = cross_val_score(pipeline, X_train, y_train, cv=tscv, scoring='accuracy', n_jobs=-1)
    logger.info(f'Scores de validação cruzada (TimeSeriesSplit) no treino: {cv_scores}')
    logger.info(f'Acurácia média da validação cruzada: {np.mean(cv_scores):.4f} (+/- {np.std(cv_scores):.4f})')

    # Treinar o pipeline no conjunto de treino completo
    pipeline.fit(X_train, y_train)
    logger.info('Pipeline treinado com sucesso no conjunto de treino completo.')

    # Capturar acurácia de treino para métricas
    y_train_pred = pipeline.predict(X_train)
    accuracy_train = accuracy_score(y_train, y_train_pred)
    logger.info(f'Acurácia no conjunto de treino: {accuracy_train:.4f}')

except Exception as e:
    logger.error(f'Erro durante a construção ou treinamento do pipeline: {e}')
    raise

2026-01-16 03:05:10,232 - INFO - Iniciando construção e treinamento do pipeline de machine learning.
INFO:__main__:Iniciando construção e treinamento do pipeline de machine learning.
2026-01-16 03:05:10,236 - INFO - Pipeline definido: StandardScaler -> LogisticRegression.
INFO:__main__:Pipeline definido: StandardScaler -> LogisticRegression.
2026-01-16 03:05:20,076 - INFO - Scores de validação cruzada (TimeSeriesSplit) no treino: [0.45909646 0.48351648 0.47496947 0.54212454 0.47008547]
INFO:__main__:Scores de validação cruzada (TimeSeriesSplit) no treino: [0.45909646 0.48351648 0.47496947 0.54212454 0.47008547]
2026-01-16 03:05:20,078 - INFO - Acurácia média da validação cruzada: 0.4860 (+/- 0.0292)
INFO:__main__:Acurácia média da validação cruzada: 0.4860 (+/- 0.0292)
2026-01-16 03:05:24,665 - INFO - Pipeline treinado com sucesso no conjunto de treino completo.
INFO:__main__:Pipeline treinado com sucesso no conjunto de treino completo.
2026-01-16 03:05:24,675 - INFO - Acurácia no conj

## 5. Avaliação do Modelo no Conjunto de Teste

In [51]:
logger.info('Iniciando avaliação do modelo no conjunto de teste.')

try:
    # Fazer predições no conjunto de teste
    y_pred = pipeline.predict(X_test)
    y_proba = pipeline.predict_proba(X_test)[:, 1] # Probabilidade da classe positiva (1)
    logger.info('Predições e probabilidades geradas para o conjunto de teste.')

    # Calcular métricas de avaliação
    test_metrics = _evaluate_model(y_test, y_pred, y_proba)
    test_metrics['accuracy_train'] = accuracy_train # Adiciona a acurácia de treino às métricas

    logger.info('Métricas de Teste:' +
                f'  Acurácia: {test_metrics['accuracy_test']:.4f}' +
                f'  Precisão: {test_metrics['precision_test']:.4f}' +
                f'  Recall: {test_metrics['recall_test']:.4f}' +
                f'  F1-Score: {test_metrics['f1_test']:.4f}' +
                f'  ROC AUC: {test_metrics['roc_auc_test']:.4f}')
    logger.info('Relatório de Classificação:' + test_metrics['classification_report'])

except Exception as e:
    logger.error(f'Erro durante a avaliação do modelo no conjunto de teste: {e}')
    raise


2026-01-16 03:05:24,686 - INFO - Iniciando avaliação do modelo no conjunto de teste.
INFO:__main__:Iniciando avaliação do modelo no conjunto de teste.
2026-01-16 03:05:24,697 - INFO - Predições e probabilidades geradas para o conjunto de teste.
INFO:__main__:Predições e probabilidades geradas para o conjunto de teste.
2026-01-16 03:05:24,699 - INFO - Calculando métricas de avaliação...
INFO:__main__:Calculando métricas de avaliação...
2026-01-16 03:05:24,720 - INFO - Métricas calculadas com sucesso.
INFO:__main__:Métricas calculadas com sucesso.
2026-01-16 03:05:24,723 - INFO - Métricas de Teste:  Acurácia: 0.6000  Precisão: 0.5000  Recall: 0.6667  F1-Score: 0.5714  ROC AUC: 0.5602
INFO:__main__:Métricas de Teste:  Acurácia: 0.6000  Precisão: 0.5000  Recall: 0.6667  F1-Score: 0.5714  ROC AUC: 0.5602
2026-01-16 03:05:24,724 - INFO - Relatório de Classificação:              precision    recall  f1-score   support

           0       0.71      0.56      0.62        18
           1       0

## 6. Exportação de Artefatos

In [52]:
logger.info('Iniciando exportação de artefatos.')

try:
    _create_directory(ARTIFACTS_DIR)

    # 1. Exportar o pipeline completo em formato pickle
    pipeline_path = os.path.join(ARTIFACTS_DIR, "logreg_pipeline.pkl")
    joblib.dump(pipeline, pipeline_path)
    logger.info(f'Pipeline salvo em: {pipeline_path}')

    # 2. Exportar as principais métricas em JSON
    metrics_path = os.path.join(ARTIFACTS_DIR, 'metrics.json')
    with open(metrics_path, 'w') as f:
        json.dump(test_metrics, f, indent=4)
    logger.info(f'Métricas salvas em: {metrics_path}')

    # 3. Exportar as predições sobre o conjunto de teste em CSV
    predictions_df = pd.DataFrame({
        'Data': data_test_series.values, # Usar a série de datas salva anteriormente
        'y_true': y_test.values,
        'y_pred': y_pred,
        'y_proba': y_proba
    })
    predictions_path = os.path.join(ARTIFACTS_DIR, "test_predictions.csv")
    predictions_df.to_csv(predictions_path, index=False)
    logger.info(f'Predições do conjunto de teste salvas em: {predictions_path}')

    logger.info('Exportação de todos os artefatos concluída com sucesso.')

except Exception as e:
    logger.error(f'Erro durante a exportação de artefatos: {e}')
    raise

2026-01-16 03:05:28,116 - INFO - Iniciando exportação de artefatos.
INFO:__main__:Iniciando exportação de artefatos.
2026-01-16 03:05:28,120 - INFO -  Diretório artifacts garantido.
INFO:__main__: Diretório artifacts garantido.
2026-01-16 03:05:28,126 - INFO - Pipeline salvo em: artifacts/logreg_pipeline.pkl
INFO:__main__:Pipeline salvo em: artifacts/logreg_pipeline.pkl
2026-01-16 03:05:28,130 - INFO - Métricas salvas em: artifacts/metrics.json
INFO:__main__:Métricas salvas em: artifacts/metrics.json
2026-01-16 03:05:28,136 - INFO - Predições do conjunto de teste salvas em: artifacts/test_predictions.csv
INFO:__main__:Predições do conjunto de teste salvas em: artifacts/test_predictions.csv
2026-01-16 03:05:28,139 - INFO - Exportação de todos os artefatos concluída com sucesso.
INFO:__main__:Exportação de todos os artefatos concluída com sucesso.
